## Barry's Bootcamp Seattle Expansion

---

Barry’s is one of the fastest growing fitness studios globally. Barry's has one studio in most of the major cities around the the world except for it's headquarters in New York City as well as the locations in Los Angeles and San Francisco. 

I will be learning the San Francisco locations with k-means clustering to understand what other popular stores are nearby to make them successful.


#### Importing Libraries
First, let's import the necessary libraies to import, analyze and visualize data.

In [1]:
#necessary packages for data analysis, clustering, and visualization
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Foursquare credentials to for the foursquare API request.

In [8]:
#foursquare credentials
CLIENT_ID = 'YR1UR50NOSUS0C200Z5NBEE5PATRMZNA51UKOMR11ETV3B0J' # your Foursquare ID
CLIENT_SECRET = 'O4JKK2KVD5YC34KHTHBLTPF4T5J2QTT14NOGK0YIRHB4D2OT' # your Foursquare Secret
VERSION = '20200218'
radius = 150
LIMIT = 50

#### Gathering San Francisco data to better understand what popular locations are around.

Gathering data with the foursquare API.

In [9]:
#Barry's FiDi location
FiDi_address = '333 Bush St, San Francisco, CA'
geolocator = Nominatim(user_agent="foursquare_agent")
FiDi_location = geolocator.geocode(FiDi_address)
FiDi_latitude = FiDi_location.latitude
FiDi_longitude = FiDi_location.longitude
print('FiDi latitude:',FiDi_latitude,'\n','FiDi longitude:',FiDi_longitude)

#Barry's Marina location
Mari_address = '2246 Lombard St, San Francisco, CA'
geolocator = Nominatim(user_agent="foursquare_agent")
Mari_location = geolocator.geocode(Mari_address)
Mari_latitude = Mari_location.latitude
Mari_longitude = Mari_location.longitude
print('Mari latitude:',Mari_latitude,'\n','Mari longitude:',Mari_longitude)

#Barry's SoMa location
SoMa_address = '236 King St, San Francisco, CA'
geolocator = Nominatim(user_agent="foursquare_agent")
SoMa_location = geolocator.geocode(SoMa_address)
SoMa_latitude = SoMa_location.latitude
SoMa_longitude = SoMa_location.longitude
print('SoMa latitude:',SoMa_latitude,'\n','SoMa longitude:',SoMa_longitude)

#Barry's Castro location
Cast_address = '2280 Market Street San Francisco, CA'
geolocator = Nominatim(user_agent="foursquare_agent")
Cast_location = geolocator.geocode(Cast_address)
Cast_latitude = Cast_location.latitude
Cast_longitude = Cast_location.longitude
print('Cast latitude:',Cast_latitude,'\n','Cast longitude:',Cast_longitude)

FiDi latitude: 37.790464650000004 
 FiDi longitude: -122.40305193623519
Mari latitude: 37.79967579591837 
 Mari longitude: -122.43850363265307
SoMa latitude: 37.777564734693875 
 SoMa longitude: -122.39282502040817
Cast latitude: 37.7648157 
 Cast longitude: -122.4327172


In [10]:
import requests

In [11]:
#foursqure output
FiDi_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, FiDi_latitude, FiDi_longitude, VERSION, radius, LIMIT)
Mari_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Mari_latitude, Mari_longitude, VERSION, radius, LIMIT)
SoMa_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, SoMa_latitude, SoMa_longitude, VERSION, radius, LIMIT)
Cast_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Cast_latitude, Cast_longitude, VERSION, radius, LIMIT)
#examine results
FiDi_results = requests.get(FiDi_url).json()
Mari_results = requests.get(Mari_url).json()
SoMa_results = requests.get(SoMa_url).json()
Cast_results = requests.get(Cast_url).json()
#get relevant columns
FiDi_items = FiDi_results['response']['groups'][0]['items']
Mari_items = Mari_results['response']['groups'][0]['items']
SoMa_items = SoMa_results['response']['groups'][0]['items']
Cast_items = Cast_results['response']['groups'][0]['items']

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

***Turning the foursquare outputs into a pandas dataframe.***

In [13]:
FiDi_dataframe = json_normalize(FiDi_items) # flatten JSON

# filter columns
FiDi_filtered_columns = ['venue.name', 'venue.categories'] + [col for col in FiDi_dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
FiDi_dataframe_filtered = FiDi_dataframe.loc[:, FiDi_filtered_columns]

# filter the category for each row
FiDi_dataframe_filtered['venue.categories'] = FiDi_dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
FiDi_dataframe_filtered.columns = [col.split('.')[-1] for col in FiDi_dataframe_filtered.columns]

FiDi_dataframe_filtered
FiDi_dataframe_filtered_Clean = FiDi_dataframe_filtered[FiDi_dataframe_filtered.name != "Barry's Bootcamp"]
FiDi_dataframe_filtered_Clean.drop(['address', 'cc', 'city', 'country','crossStreet','formattedAddress','neighborhood', 'postalCode', 'state','labeledLatLngs'],axis=1,inplace=True)
FiDi_dataframe_filtered_Clean.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,name,categories,distance,lat,lng,id
0,Pushkin,Russian Restaurant,90,37.790943,-122.403877,58cc4d2fe0adac17bbf4838e
1,Akiko’s Restaurant & Sushi Bar,Sushi Restaurant,142,37.790623,-122.404657,49e7e380f964a52027651fe3
2,Sababa,Mediterranean Restaurant,130,37.791185,-122.404219,5746063b498e3939498157c9
3,Satori Yoga Studio,Yoga Studio,69,37.790098,-122.402417,4acbef02f964a5205fc820e3
4,Blue Bottle Coffee,Coffee Shop,96,37.789744,-122.403652,5daddd28a4f2c900078367c5


In [18]:
Mari_dataframe = json_normalize(Mari_items) # flatten JSON

# filter columns
Mari_filtered_columns = ['venue.name', 'venue.categories'] + [col for col in Mari_dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
Mari_dataframe_filtered = Mari_dataframe.loc[:, Mari_filtered_columns]

# filter the category for each row
Mari_dataframe_filtered['venue.categories'] = Mari_dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
Mari_dataframe_filtered.columns = [col.split('.')[-1] for col in Mari_dataframe_filtered.columns]

Mari_dataframe_filtered

Mari_dataframe_filtered_Clean = Mari_dataframe_filtered[Mari_dataframe_filtered.name != "Barry's Bootcamp"]
Mari_dataframe_filtered_Clean.drop(['address', 'cc', 'city', 'country','crossStreet','formattedAddress','neighborhood', 'postalCode', 'state','labeledLatLngs'],axis=1,inplace=True)
Mari_dataframe_filtered_Clean.head()

,name,categories,distance,lat,lng,id
1,SusieCakes,Cupcake Shop,101,37.800546,-122.438142,4c27c151ed0ac9b6ee3a5faa
2,Delarosa,Pizza Place,86,37.800287,-122.439110,4b0d8fe5f964a5201e4b23e3
3,Lucca's Delicatessen,Deli / Bodega,101,37.800583,-122.438362,4a620c4df964a520e7c21fe3
4,Home Plate,Diner,128,37.799795,-122.437051,4a9ec745f964a520263b20e3
5,Blue Barn Gourmet,Salad Place,113,37.800328,-122.437516,59f8cc306fa81f0ce77666ec


In [19]:
SoMa_dataframe = json_normalize(SoMa_items) # flatten JSON

# filter columns
Soma_filtered_columns = ['venue.name', 'venue.categories'] + [col for col in SoMa_dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
SoMa_dataframe_filtered = SoMa_dataframe.loc[:, Soma_filtered_columns]

# filter the category for each row
SoMa_dataframe_filtered['venue.categories'] = SoMa_dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
SoMa_dataframe_filtered.columns = [col.split('.')[-1] for col in SoMa_dataframe_filtered.columns]

SoMa_dataframe_filtered_Clean = SoMa_dataframe_filtered[SoMa_dataframe_filtered.name != "Barry's Bootcamp"]
SoMa_dataframe_filtered_Clean.drop(['address', 'cc', 'city', 'country','crossStreet','formattedAddress','postalCode', 'state','labeledLatLngs'],axis=1,inplace=True)
SoMa_dataframe_filtered_Clean.head()

,name,categories,distance,lat,lng,neighborhood,id
0,Cafe Okawari,Café,108,37.778169,-122.393798,NaN,5cddccf91822230025707c7a
1,Petit Marlowe,Wine Bar,99,37.778101,-122.393727,NaN,5941dd4ae2ead1688f35f615
3,Cento,Coffee Shop,114,37.778140,-122.393900,SoMa,5ca6aa2db25fee0039757530
4,Tan Bella,Tanning Salon,69,37.777007,-122.393178,NaN,4b09762ef964a520a41723e3
5,District,Wine Bar,111,37.778514,-122.393237,NaN,461604b8f964a52045451fe3


In [20]:
Cast_dataframe = json_normalize(Cast_items) # flatten JSON

# filter columns
Cast_filtered_columns = ['venue.name', 'venue.categories'] + [col for col in Cast_dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
Cast_dataframe_filtered = Cast_dataframe.loc[:, Cast_filtered_columns]

# filter the category for each row
Cast_dataframe_filtered['venue.categories'] = Cast_dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
Cast_dataframe_filtered.columns = [col.split('.')[-1] for col in Cast_dataframe_filtered.columns]

Cast_dataframe_filtered_Clean = Cast_dataframe_filtered[Cast_dataframe_filtered.name != "Barry's Bootcamp"]
Cast_dataframe_filtered_Clean.drop(['address', 'cc', 'city', 'country','crossStreet','formattedAddress','postalCode', 'state','labeledLatLngs'],axis=1,inplace=True)
Cast_dataframe_filtered_Clean.head()

,name,categories,distance,lat,lng,id
1,Alex Fitness,Gym,49,37.764689,-122.432183,4eaa159c7ee5cd8bcd5317c8
2,Ritual Coffee Roasters,Coffee Shop,87,37.764122,-122.433183,5925a9a1a306194f4f8a4d93
3,Dinosaurs,Sandwich Place,73,37.764403,-122.432071,4c393e38dfb0e21e2a3ab0a8
4,Super Duper Burgers,Burger Joint,123,37.764024,-122.433703,4bc38e68abf49521beedc393
5,Hi Tops,Gay Bar,80,37.764976,-122.431826,50baba7fe4b0cdd4bc1cf0ba


In [21]:
#combine dataframes of locations around barry's
dataframes_clean = [FiDi_dataframe_filtered_Clean,Mari_dataframe_filtered_Clean,SoMa_dataframe_filtered_Clean,Cast_dataframe_filtered_Clean]
Combined_dataframes_filtered_Clean = pd.concat(dataframes_clean, ignore_index=True, sort=False, keys=['FiDi', 'Mari','Soma','Cast'])

#combine all of the barry's locations
FiDi_dataframe_barrys = FiDi_dataframe_filtered[FiDi_dataframe_filtered.name == "Barry's Bootcamp"]
Mari_dataframe_barrys = Mari_dataframe_filtered[Mari_dataframe_filtered.name == "Barry's Bootcamp"]
SoMa_dataframe_barrys = SoMa_dataframe_filtered[SoMa_dataframe_filtered.name == "Barry's Bootcamp"]
Cast_dataframe_barrys = Cast_dataframe_filtered[Cast_dataframe_filtered.name == "Barry's Bootcamp"]
dataframes_barrys = [FiDi_dataframe_barrys,Mari_dataframe_barrys,SoMa_dataframe_barrys,Cast_dataframe_barrys]
Combined_dataframe_barrys = pd.concat(dataframes_barrys, ignore_index=True, sort=False, keys=['FiDi', 'Mari','Soma','Cast'])
Combined_dataframe_barrys

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Barry's Bootcamp,Gym / Fitness Center,333 Bush St,US,San Francisco,United States,NaN,26,"[333 Bush St, San Francisco, CA 94104, United ...","[{'label': 'display', 'lat': 37.790608, 'lng':...",37.790608,-122.402807,NaN,94104,CA,58e7ea770593e114d3b1705e
1,Barry's Bootcamp,Gym / Fitness Center,2246 Lombard St,US,San Francisco,United States,NaN,21,"[2246 Lombard St, San Francisco, CA 94123, Uni...","[{'label': 'display', 'lat': 37.79975842580082...",37.799758,-122.438723,NaN,94123,CA,56758f43498e50c50235bc4e
2,Barry's Bootcamp,Gym / Fitness Center,236 King St,US,San Francisco,United States,btwn 3rd & 4th St,26,"[236 King St (btwn 3rd & 4th St), San Francisc...","[{'label': 'display', 'lat': 37.777783, 'lng':...",37.777783,-122.392703,NaN,94107,CA,538a5388498eed7dcdb0ec82
3,Barry's Bootcamp,Gym / Fitness Center,2280 Market St,US,San Francisco,United States,Noe St,10,"[2280 Market St (Noe St), San Francisco, CA 94...","[{'label': 'display', 'lat': 37.76474182764215...",37.764742,-122.432785,NaN,94114,CA,5c8b072228374e002c80131a


***Let's take a look at the combined top locations around the Barry's.***

In [22]:
Combined_dataframes_filtered_Clean.head()

,name,categories,distance,lat,lng,id,neighborhood
0,Pushkin,Russian Restaurant,90,37.790943,-122.403877,58cc4d2fe0adac17bbf4838e,NaN
1,Akiko’s Restaurant & Sushi Bar,Sushi Restaurant,142,37.790623,-122.404657,49e7e380f964a52027651fe3,NaN
2,Sababa,Mediterranean Restaurant,130,37.791185,-122.404219,5746063b498e3939498157c9,NaN
3,Satori Yoga Studio,Yoga Studio,69,37.790098,-122.402417,4acbef02f964a5205fc820e3,NaN
4,Blue Bottle Coffee,Coffee Shop,96,37.789744,-122.403652,5daddd28a4f2c900078367c5,NaN


***Visualize the Barrry's locations and the top locations around them in San Francisco.***

In [15]:
#getting the coordinate of San Francisco
address = 'San Francisco'

SFO_geolocator = Nominatim(user_agent="SFO_explorer")
location = SFO_geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


In [23]:
# create map of San Francisco using latitude and longitude values
map_SFO = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, name, categories in zip(Combined_dataframes_filtered_Clean['lat'], Combined_dataframes_filtered_Clean['lng'], Combined_dataframes_filtered_Clean['name'], Combined_dataframes_filtered_Clean['categories']):
    label = '{},{}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SFO)
    
for lat, lng, name, categories in zip(Combined_dataframe_barrys['lat'], Combined_dataframe_barrys['lng'], Combined_dataframe_barrys['name'], Combined_dataframe_barrys['categories']):
    label = '{},{}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3139',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SFO)  
    
map_SFO

***Quick analysis of the top categories of the venues around the Barry's locations.***

In [48]:
#quickly look at the top locations' categories near the barry's, these are the locations we will want to target around our new possible locations to open our second location in Seattle.  
pd.get_dummies(Combined_dataframes_filtered_Clean['categories'])
top_categories = Combined_dataframes_filtered_Clean['categories'].value_counts(ascending=False)
top_categories.head(8)

Coffee Shop          6
French Restaurant    6
Gym                  6
Sushi Restaurant     5
Sandwich Place       5
Burger Joint         4
Wine Bar             3
Salad Place          3
Name: categories, dtype: int64

***Now that we understand the top locations around the barry's locations, we will explore our possible neighborhoods in Seattle, WA***

In [25]:
#Capitol Hill Neighborhoods
CapHill_address = 'Capitol Hill, Seattle, WA'
geolocator = Nominatim(user_agent="WA_foursquare_agent")
CapHill_location = geolocator.geocode(CapHill_address)
CapHill_latitude = CapHill_location.latitude
CapHill_longitude = CapHill_location.longitude
print('CapHill latitude:',CapHill_latitude,'\n','CapHill longitude:',CapHill_longitude)


CapHill latitude: 47.6238307 
 CapHill longitude: -122.3183689


In [46]:
#generate the foursquare api URLs for Capitol Hill and Ballard
#setting new raidus and limit becuase we want to search in a much larger locations
radius_SEA = 1000
LIMIT_SEA = 1000

#Capitol Hill trending locations
url_CapHill = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, CapHill_latitude, CapHill_longitude, VERSION, radius_SEA, LIMIT_SEA)
print(url_CapHill)

https://api.foursquare.com/v2/venues/explore?client_id=YR1UR50NOSUS0C200Z5NBEE5PATRMZNA51UKOMR11ETV3B0J&client_secret=O4JKK2KVD5YC34KHTHBLTPF4T5J2QTT14NOGK0YIRHB4D2OT&ll=47.6238307,-122.3183689&v=20200218&radius=1000&limit=1000


In [37]:
#generate Capitol Hill data
results_CapHill = requests.get(url_CapHill).json()

In [47]:
CapHill_items = results_CapHill['response']['groups'][0]['items']
CapHill_df = json_normalize(CapHill_items)

# filter columns
CapHill_filtered_columns = ['venue.name', 'venue.categories'] + [col for col in CapHill_df.columns if col.startswith('venue.location.')] + ['venue.id']
CapHill_df_filtered = CapHill_df.loc[:, CapHill_filtered_columns]

# filter the category for each row
CapHill_df_filtered['venue.categories'] = CapHill_df_filtered.apply(get_category_type, axis=1)

# clean columns
CapHill_df_filtered.columns = [col.split('.')[-1] for col in CapHill_df_filtered.columns]

#select the top categories
CapHill_df_filtered_Clean1 = CapHill_df_filtered[CapHill_df_filtered.categories == 'Gym']
CapHill_df_filtered_Clean2 = CapHill_df_filtered[CapHill_df_filtered.categories == 'French Restaurant']
CapHill_df_filtered_Clean3 = CapHill_df_filtered[CapHill_df_filtered.categories == 'Coffee Shop']
CapHill_df_filtered_Clean4 = CapHill_df_filtered[CapHill_df_filtered.categories == 'Sushi Restaurant']
CapHill_df_filtered_Clean5 = CapHill_df_filtered[CapHill_df_filtered.categories == 'Sandwich Place']
CapHill_df_filtered_Clean6 = CapHill_df_filtered[CapHill_df_filtered.categories == 'Salad Place']
CapHill_df_filtered_Clean7 = CapHill_df_filtered[CapHill_df_filtered.categories == 'Burger Joint']
CapHill_df_filtered_Clean8 = CapHill_df_filtered[CapHill_df_filtered.categories == 'Café']

#combine dataframes of top locations
CapHill_Combined = [CapHill_df_filtered_Clean1,CapHill_df_filtered_Clean2,CapHill_df_filtered_Clean3,CapHill_df_filtered_Clean4,CapHill_df_filtered_Clean5,
                   CapHill_df_filtered_Clean6,CapHill_df_filtered_Clean7,CapHill_df_filtered_Clean8]
CapHill_Combined_df = pd.concat(CapHill_Combined, ignore_index=True, sort=False)
CapHill_Combined_df.drop(['address', 'cc', 'city', 'country','crossStreet','formattedAddress','postalCode', 'state','labeledLatLngs'],axis=1,inplace=True)
CapHill_Combined_df_analyze = CapHill_Combined_df[['name','lat','lng']]

CapHill_Combined_df_analyze


,name,lat,lng
0,TRIBE Fitness,47.624561,-122.320756
1,Pro Sports Club,47.623450,-122.329550
2,Espresso Vivace,47.623667,-122.320692
3,Espresso Vivace Sidewalk Bar,47.621634,-122.321057
4,Joe Bar,47.625108,-122.321621
5,Victrola,47.622364,-122.312907
6,Analog Coffee,47.620768,-122.325448
7,Caffe Ladro,47.623013,-122.312765
8,Espresso Vivace,47.620232,-122.330508
9,Taneda Sushi In Kaiseki,47.620583,-122.321300


***Import the necessary libraries for normalizing the data and k-means clustering.***

In [40]:
from sklearn.preprocessing import StandardScaler
X_CapHill = CapHill_Combined_df_analyze.values[:,1:]
X_CapHill = np.nan_to_num(X_CapHill)
Clus_dataSet_CapHill = StandardScaler().fit_transform(X_CapHill)
Clus_dataSet_CapHill

from sklearn.cluster import KMeans

clusterNum = 5
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(X_CapHill)
labels_CapHill = k_means.labels_

CapHill_Combined_df_analyze["Clus_km"] = labels_CapHill
k_means_cluster_centroids_CapHill = CapHill_Combined_df_analyze.groupby('Clus_km').mean()

k_means_cluster_centroids_CapHill.reset_index(inplace=True)
k_means_cluster_centroids_CapHill

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Clus_km,lat,lng
0,0,47.624446,-122.321023
1,1,47.620578,-122.322220
2,2,47.623128,-122.328936
3,3,47.622689,-122.312836
4,4,47.614886,-122.319493


***Visulize our centroids (recommended locations) for a new Barry's studio.***

In [41]:
#getting the coordinate of Seattle
address_sea = 'Seattle'

sea_geolocator = Nominatim(user_agent="sea_explorer")
location_sea = sea_geolocator.geocode(address_sea)
latitude_sea = location_sea.latitude
longitude_sea = location_sea.longitude

# create map of Seattle using latitude and longitude values
map_sea = folium.Map(location=[latitude_sea, longitude_sea], zoom_start=14)

# add markers to map
for lat, lng, name, categories in zip(CapHill_Combined_df['lat'], CapHill_Combined_df['lng'], CapHill_Combined_df['name'], CapHill_Combined_df['categories']):
    label = '{},{}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sea)
    
for lat, lng, Clus_km in zip(k_means_cluster_centroids_CapHill['lat'], k_means_cluster_centroids_CapHill['lng'], k_means_cluster_centroids_CapHill['Clus_km']):
    label = '{},{}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3139',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sea)  
    
map_sea

The geograpical coordinate of Seattle are 37.7790262, -122.4199061.
